# Assignment 1 (Coding Problems)

In this assignment, you will implement the operators we have discussed in class. This will help you get a deeper understanding of the computation in each operator.

## Overview of the problems
- Problem 1 (20 pts): Matrix multiplication
- Problem 2 (50 pts): CONV2D without padding and stride
- Problem 3 (30 pts): CONV2D with stride but without padding

In [1]:
# Please do not modify this cell
import torch
from typing import Tuple
import random

filter_resolution = [1, 3, 5, 7]
channel_counts = [1, 3, 16]
stride_amounts = [2, 3, 4]



def test_res_printer(
    test_res: Tuple[int],
    test_name: str = "Test"
) -> None:
    print("<Test {} Results>".format(test_name))
    print("- Correct: {}".format(test_res[0]))
    print("- Incorrect: {}".format(test_res[1]))
    print("-> Passed {}% of tests".format((test_res[0]/test_res[2])*100))

## Example code: Scalar - Vector Multiplication
I provide this code to help you get familiar with coding in Python.

In [2]:
# Note: Python works without specifying types, but specifying the input and output types is recommended when
# you work with a cross-stack development team.
#
# Benefit 1: Enhances the code readability; people eventually need to understand the types when
#            trying to understand the code. It's better to explicityly provide the types instead of letting
#            people spend their time to guess them.
#
# Benefit 2: Type information is necessary for compilers and other tools. Explicit information can make your
#            implementation easier for tools to understand it.

def scalar_vector_product (
    input_scalar: torch.double, # Default data type in Pytorch (double == float64)
    input_vector: torch.Tensor, # Unlike C++, you can leave the last comma even if it is the last argument
) -> torch.Tensor:
    # Input validity check
    assert(
        len(input_vector.size()) == 1 # Is input_vector a 1D tensor?
        and input_vector.size()[0] > 0 # Does input_vector contain at least one element?
    )

    output_vector = input_scalar * input_vector

    return output_vector


The following cell is a test to validate the implementation, comparing the results against Pytorch's implementation

In [3]:
# Test program: Please do not modify this cell
def test_example(number_of_tests: int) -> None:
    example_correct = 0
    example_incorrect = 0

    for test_id in range(number_of_tests):
        vector_length = torch.randint(2, 256,[1,])[0]
        input_scalar = torch.rand(1)
        input_vector = torch.randn(vector_length)

        test_output = scalar_vector_product(input_scalar, input_vector)
        reference_output = torch.mul(input_vector, input_scalar)

        if torch.allclose(test_output, reference_output, atol=1e-6):
            example_correct += 1
        else:
            example_incorrect += 1

    return (example_correct, example_incorrect, number_of_tests)


In [4]:
# Run the test
test_example_res = test_example(10)
test_res_printer(test_example_res, "Example")


<Test Example Results>
- Correct: 10
- Incorrect: 0
-> Passed 100.0% of tests


## Problem 1 (20 pts)

Implement a matrix multiplication that matches the behavior of "torch.matmul" in Pytorch.

(reference: https://pytorch.org/docs/stable/generated/torch.matmul.html)



In [5]:
def my_matmul (
    matrix_A: torch.Tensor, # M x K matrix
    matrix_B:torch.Tensor # K x N matrix
) -> torch.Tensor:

    # raise NotImplementedError # Delete this line when you start implementation

    # The following lines extract tensor dimension information
    M = matrix_A.size()[0]
    K = matrix_B.size()[0]
    N = matrix_B.size()[1]

    output_matrix = torch.zeros((M,N))

    # Implement here
    for cid in range(N):
        for rid, row in enumerate(matrix_A):
          col = matrix_B[:, cid]
          assert len(row) == len(col)
          temp = 0.0
          for id, ele in enumerate(row):
              temp += ele * col[id]
          output_matrix[rid, cid] = temp
    return output_matrix

In [6]:
# Test program: Please do not modify this cell
def test_matmul(num_tests: int = 10):
    correct = 0
    incorrect = 0

    for test_id in range(num_tests):
        problem_size = torch.randint(2, 32,[3,])
        M = problem_size[0]
        N = problem_size[1]
        K = problem_size[2]

        matrix_A = torch.randn(M,K)
        matrix_B = torch.randn(K,N)

        reference_matrix_C = torch.matmul(matrix_A, matrix_B)
        test_matrix_C = my_matmul(matrix_A, matrix_B)

        if torch.allclose(reference_matrix_C, test_matrix_C, atol=1e-6):
            correct += 1
        else:
            incorrect += 1

    return (correct, incorrect, num_tests)

In [8]:
test_res = test_matmul(10)
test_res_printer(test_res, "Problem 1")

<Test Problem 1 Results>
- Correct: 10
- Incorrect: 0
-> Passed 100.0% of tests


## Problem 2 (50 pts)
Implement a conv2d without padding and stride. The results should match the behavior of "torch.conv2d" in Pytorch.

(reference: https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)

In [12]:
def my_conv2d(
    weight: torch.Tensor, # [K, C, R, S]
    input_activation: torch.Tensor, # [1, C, H, W]
) -> torch.Tensor: # [1, K, P, Q]

    # raise NotImplementedError # Delete this line when you start implementation

    K, C, R, S = weight.size()
    # K = weight.size()[0]
    # C = weight.size()[1]
    # ...
    B, C_, H, W = input_activation.size()
    assert C == C_
    P = H - R + 1
    Q = W - S + 1

    output_activation = torch.zeros([B, K, P, Q])

    # Implement here
    for b in range(B):
        # for each output feature maps
        for k in range(K):
            # for each output element
            for p in range(P):
                for q in range(Q):
                    output_activation[b, k, p, q] = 0.0
                    for c in range(C):
                        for r in range(R):
                            for s in range(S):
                                output_activation[b, k, p, q] += \
                                input_activation[b, c, p + r, q + s] * weight[k, c, r, s]


    return output_activation

In [13]:
# Test program: Please do not modify this cell
def test_conv2d(num_tests: int = 3):
    correct = 0
    incorrect = 0

    for test_id in range(num_tests):
        activaiton_resolution = torch.randint(7, 32,[2,])

        K = int(random.choice(channel_counts))
        C = int(random.choice(channel_counts))
        H = activaiton_resolution[0]
        W = activaiton_resolution[1]
        R = int(random.choice(filter_resolution))
        S = R

        if H < R:
            H += R

        if W < S:
            W += S

        # Generate random input activation and weight
        weight = torch.randn(K,C,R,S)
        input_activation = torch.randn(1,C,H,W)

        # Compute the reference output
        reference_conv = torch.nn.Conv2d(C, K, (R,S), bias=False)
        reference_conv.weight = torch.nn.Parameter(weight)

        reference_output = reference_conv(input_activation)

        # Compute the test output
        test_output = my_conv2d(weight, input_activation)

        if torch.allclose(reference_output, test_output,  atol=1e-3):
            correct += 1
        else:
            incorrect += 1
        print("Finished test run {}".format(test_id))

    return (correct, incorrect, num_tests)

In [14]:
test_res = test_conv2d(3)
test_res_printer(test_res, "Problem 2")

Finished test run 0
Finished test run 1
Finished test run 2
<Test Problem 2 Results>
- Correct: 3
- Incorrect: 0
-> Passed 100.0% of tests


## Problem 3 (30 pts)
Using the answer of problem 2, implement a strided convolution. We assume the same stride amount in vertical and horizontal (H and W in our diagrams) dimensions

In [37]:
def my_conv2d_with_stride(
    weight: torch.Tensor, # [K, C, R, S]
    input_activation: torch.Tensor, # [1, C, H, W],
    stride: int
) -> torch.Tensor: # [1, K, P, Q]

    # raise NotImplementedError # Delete this line when you start implementation

    K, C, R, S = weight.size()
    B, C_, H, W = input_activation.size()
    assert C == C_
    P = (H - R) // stride + 1
    Q = (W - S) // stride + 1
    output_activation = torch.zeros([B, K, P, Q])

    # print('weight: KCRS=', weight.size())
    # print('input: BCHW=', input_activation.size())
    # print(stride)
    # print('output: BKPQ=', output_activation.size())
    # Implemente here
    # Hint: output_activation = torch.zeros([1,K,P,Q])

    # Implement here
    for b in range(B):
        # for each output feature maps
        for k in range(K):
            # for each output element
            for p in range(P):
                for q in range(Q):
                    output_activation[b, k, p, q] = 0.0;
                    for c in range(C):
                        for r in range(R):
                            for s in range(S):
                                output_activation[b, k, p, q] += \
                                input_activation[b, c, p * stride + r, q * stride + s] * weight[k, c, r, s];
    return output_activation

In [38]:
# Test program: Please do not modify this cell
def test_conv2d_with_stride(num_tests: int = 3):
    correct = 0
    incorrect = 0

    for test_id in range(num_tests):
        activaiton_resolution = torch.randint(7, 32,[2,])
        stride_amount = int(random.choice(stride_amounts))
        K = int(random.choice(channel_counts))
        C = int(random.choice(channel_counts))
        H = activaiton_resolution[0]
        W = activaiton_resolution[1]
        R = int(random.choice(filter_resolution))
        S = R

        if H < R:
            H += R

        if W < S:
            W += S

        # Generate random input activation and weight
        weight = torch.randn(K,C,R,S)
        input_activation = torch.randn(1,C,H,W)

        # Compute the reference output
        reference_conv = torch.nn.Conv2d(C, K, (R,S), stride = (stride_amount,stride_amount), bias = False)
        reference_conv.weight = torch.nn.Parameter(weight)

        reference_output = reference_conv(input_activation)
        # print('refer: BKPQ=', reference_output.size())

        # Compute the test output
        test_output = my_conv2d_with_stride(weight, input_activation, stride_amount)

        if torch.allclose(reference_output, test_output,  atol=1e-3):
            correct += 1
        else:
            incorrect += 1
        print("Finished test run {}".format(test_id))

    return (correct, incorrect, num_tests)

In [39]:
test_res = test_conv2d_with_stride(3)
test_res_printer(test_res, "Problem 3")

Finished test run 0
Finished test run 1
Finished test run 2
<Test Problem 3 Results>
- Correct: 3
- Incorrect: 0
-> Passed 100.0% of tests
